In [4]:
# Import pandas, numpy, matplotlib, plotly, and seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [5]:
log_data = pd.read_csv('/Users/RICARD/Downloads/log_fw_3.csv', sep=';', header=None)

In [6]:
log_data.columns = ['timestamp', 'IPsrc', 'IPdst', 'proto', 'lenght', 'portdst', 'rule','action','7', '8','9']

log_data.head()

,timestamp,IPsrc,IPdst,proto,lenght,portdst,rule,action,7,8,9
0,2023-02-12 03:59:03,66.249.69.180,17.17.17.17,TCP,57630.0,443.0,1.0,PERMIT,eth0,NaN,6.0
1,2023-02-12 03:59:04,66.249.69.178,17.17.17.17,TCP,63808.0,443.0,1.0,PERMIT,eth0,NaN,6.0
2,2023-02-12 03:59:08,221.11.125.141,17.17.17.17,TCP,43123.0,2376.0,999.0,DENY,eth0,NaN,6.0
3,2023-02-12 03:59:18,183.136.225.42,17.17.17.17,TCP,8088.0,70.0,999.0,DENY,eth0,NaN,6.0
4,2023-02-12 03:59:20,38.32.112.34,17.17.17.17,TCP,6356.0,443.0,1.0,PERMIT,eth0,NaN,6.0


In [7]:
# Convert timestamp to datetime

def to_datetime(x):
    return pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S.%f')

In [8]:
log_data.timestamp = to_datetime(log_data.timestamp)
log_data.portdst = log_data.portdst.astype('object')
log_data.dtypes

timestamp    datetime64[ns]
IPsrc                object
IPdst                object
proto                object
lenght              float64
portdst              object
rule                float64
action               object
7                    object
8                   float64
9                   float64
dtype: object

In [9]:
log_data.isna().sum()

timestamp          0
IPsrc              2
IPdst              2
proto              2
lenght             2
portdst            2
rule               2
action             2
7                  2
8            1000000
9                  2
dtype: int64

In [10]:
# Regroup by IPsrc and count the total number of of requests

IPsrc_grouped = log_data.groupby('IPsrc')['IPdst'].count().sort_values(ascending=False)
IPsrc_grouped = pd.DataFrame([IPsrc_grouped.index,IPsrc_grouped]).transpose()
IPsrc_grouped.columns = ['IPsrc', 'count']

IPsrc_grouped.head(10)

,IPsrc,count
0,109.234.162.235,542741
1,198.46.189.193,61889
2,89.248.165.195,19280
3,157.90.182.23,12220
4,89.248.163.70,10725
5,45.227.253.106,3434
6,206.189.114.5,2970
7,66.249.72.202,2374
8,136.243.228.177,1912
9,3.211.110.80,1797


In [11]:
# Regroup by IPsrc and count the total number of requests denied

IPsrc_deny = log_data[log_data.action == 'DENY'].groupby('IPsrc')['IPdst'].count().sort_values(ascending=False)

IPsrc_deny = pd.DataFrame([IPsrc_deny.index,IPsrc_deny]).transpose()

IPsrc_deny.columns = ['IPsrc', 'count']

IPsrc_deny.head(10)

,IPsrc,count
0,198.46.189.193,60203
1,89.248.165.195,19280
2,89.248.163.70,10723
3,45.227.253.106,3434
4,206.189.114.5,2969
5,91.240.118.225,1700
6,79.124.62.78,1511
7,79.124.62.86,1465
8,78.128.114.78,1449
9,79.124.62.82,1288


In [12]:
# Regroup by IPsrc and count the total number of requests denied

IPsrc_permit = log_data[log_data.action == 'PERMIT'].groupby('IPsrc')['IPdst'].count().sort_values(ascending=False)

IPsrc_permit = pd.DataFrame([IPsrc_permit.index,IPsrc_permit]).transpose()

IPsrc_permit.columns = ['IPsrc', 'count']

IPsrc_permit.head(10)

,IPsrc,count
0,109.234.162.235,542741
1,157.90.182.23,12220
2,66.249.72.202,2374
3,136.243.228.177,1912
4,3.211.110.80,1797
5,66.249.72.204,1722
6,198.46.189.193,1686
7,66.249.77.24,1653
8,66.249.68.3,1643
9,44.227.118.60,1583


In [13]:
# Total number of requests by IPsrc, by action

Merge_IPsrc = pd.merge(IPsrc_grouped,IPsrc_deny, on='IPsrc', how='outer')
Merge_IPsrc = pd.merge(Merge_IPsrc,IPsrc_permit, on='IPsrc', how='outer')

Merge_IPsrc.columns = ['IPsrc', 'total', 'deny', 'permit']
Merge_IPsrc['deny'] = Merge_IPsrc['deny'].fillna(0)
Merge_IPsrc['permit'] = Merge_IPsrc['permit'].fillna(0)
Merge_IPsrc['total'] = Merge_IPsrc.total.astype('int64')

Merge_IPsrc.head(10)

,IPsrc,total,deny,permit
0,109.234.162.235,542741,0,542741
1,198.46.189.193,61889,60203,1686
2,89.248.165.195,19280,19280,0
3,157.90.182.23,12220,0,12220
4,89.248.163.70,10725,10723,2
5,45.227.253.106,3434,3434,0
6,206.189.114.5,2970,2969,1
7,66.249.72.202,2374,0,2374
8,136.243.228.177,1912,0,1912
9,3.211.110.80,1797,0,1797


In [14]:
Merge_IPsrc.dtypes

IPsrc     object
total      int64
deny       int64
permit     int64
dtype: object

In [15]:
px.pie(Merge_IPsrc.head(10), values='total', names='IPsrc', title='Total number of requests by IPsrc')

In [16]:
px.pie(Merge_IPsrc.head(10), values='deny', names='IPsrc', title='Total number of requests denied by IPsrc')

In [17]:
px.pie(Merge_IPsrc.head(10), values='permit', names='IPsrc', title='Total number of requests permited by IPsrc')

In [23]:
fig = px.scatter(Merge_IPsrc, x='permit', y='deny', labels='IPsrc', title='Total number of requests by IPsrc')

fig

In [26]:
Merge_IPsrc['ratio'] = Merge_IPsrc['deny'] / Merge_IPsrc['total']


Merge_IPsrc.head(10)    

,IPsrc,total,deny,permit,ratio
0,109.234.162.235,542741,0,542741,0.000000
1,198.46.189.193,61889,60203,1686,0.972758
2,89.248.165.195,19280,19280,0,1.000000
3,157.90.182.23,12220,0,12220,0.000000
4,89.248.163.70,10725,10723,2,0.999814
5,45.227.253.106,3434,3434,0,1.000000
6,206.189.114.5,2970,2969,1,0.999663
7,66.249.72.202,2374,0,2374,0.000000
8,136.243.228.177,1912,0,1912,0.000000
9,3.211.110.80,1797,0,1797,0.000000
